# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [76]:
import pyspark
print(pyspark.__version__)

3.5.1


In [77]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

In [78]:
raw_data_url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'

### Step 3. Assign it to a variable called chipo.

In [79]:
import requests

response = requests.get(raw_data_url)
with open('chipotle.tsv', 'wb') as f:
    f.write(response.content)

In [80]:
spark = SparkSession.builder.appName('ChipotleAnalysis').getOrCreate()
chipo = spark.read.csv('chipotle.tsv', sep='\t', header=True, inferSchema=True)

### Step 4. See the first 10 entries

In [81]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [82]:
print('Row count :', chipo.count())

Row count : 4622


### Step 6. What is the number of columns in the dataset?

In [83]:
print('Column count :', len(chipo.columns))

Column count : 5


### Step 7. Print the name of all the columns.

In [84]:
chipo.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

In [85]:
# NA in pyspark

### Step 9. Which was the most-ordered item?

In [86]:
chipo.groupBy('item_name').agg({"quantity":"sum"}).orderBy('sum(quantity)', ascending=False).show(1)

+------------+-------------+
|   item_name|sum(quantity)|
+------------+-------------+
|Chicken Bowl|          761|
+------------+-------------+
only showing top 1 row



### Step 10. For the most-ordered item, how many items were ordered?

In [87]:
chipo.groupBy('item_name').agg({"quantity":"sum"}).orderBy('sum(quantity)', ascending=False).select('sum(quantity)').show(1)

+-------------+
|sum(quantity)|
+-------------+
|          761|
+-------------+
only showing top 1 row



### Step 11. What was the most ordered item in the choice_description column?

In [88]:
chipo.groupBy('choice_description').agg({"quantity":"sum"}).orderBy('sum(quantity)', ascending=False).show(2)

+------------------+-------------+
|choice_description|sum(quantity)|
+------------------+-------------+
|              NULL|         1382|
|       [Diet Coke]|          159|
+------------------+-------------+
only showing top 2 rows



### Step 12. How many items were orderd in total?

In [89]:
chipo.agg({"quantity":"sum"}).show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [92]:
chipo.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



#### Step 13.b. Create a lambda function and change the type of item price

In [96]:
chipo = chipo.withColumn('item_price', F.regexp_replace('item_price', '$', '').cast('float'))
chipo.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



#### Step 13.c. Check the item price type

In [97]:
chipo.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



### Step 14. How much was the revenue for the period in the dataset?

In [99]:
chipo.agg({'item_price':'sum'}).show()

+------------------+
|   sum(item_price)|
+------------------+
|34500.159754514694|
+------------------+



### Step 15. How many orders were made in the period?

In [100]:
chipo.select('order_id').distinct().count()

1834

### Step 16. What is the average revenue amount per order?

In [105]:
chipo.groupBy('order_id').agg(F.round(F.avg('item_price'), 2).alias('avg_item_price')).orderBy('order_id').show()

+--------+--------------+
|order_id|avg_item_price|
+--------+--------------+
|       1|          2.89|
|       2|         16.98|
|       3|          6.33|
|       4|          10.5|
|       5|          6.85|
|       6|          8.75|
|       7|          7.85|
|       8|          5.44|
|       9|          5.33|
|      10|           6.6|
|      11|          6.19|
|      12|          7.18|
|      13|          5.44|
|      14|          5.04|
|      15|          5.44|
|      16|          5.34|
|      17|          5.04|
|      18|          6.22|
|      19|          6.95|
|      20|          9.05|
+--------+--------------+
only showing top 20 rows



### Step 17. How many different items are sold?

In [106]:
chipo.select('item_name').distinct().count()

50